In [1]:
# universally modules
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm
sys.path.append("../src")

# preprocessing and transformation modules
import fasttext
import Preprocessing
from Features import buildFeatures
from Modelling import StackingModelling
from Transformation import StackedTransformation
from ModelSelection import ModelSelection, process_case

# Scikit-Learn
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.feature_extraction.text import TfidfVectorizer

# model algorithm
from sklearn.svm import LinearSVC, LinearSVR
from xgboost import XGBRegressor, XGBClassifier
from sklearn.linear_model import SGDClassifier, SGDRegressor

# evaluation modules
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

/home/bigdata/anaconda3/lib/python3.8/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.5) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


### Parameters

In [2]:
# to speed up the process choose a sample size to randomly draw a sample of the whole daataset
sample_size = 100 

# remove all text that contain less than n chars
min_chars_per_text = 50

# which features will be used for the TF-IDF transformation
text_features = "text_preprocessed"

#### define the target variable and categorial variables used in later transformations ###
  
#### Case 1: gender 
target_variable = "gender"
categorial_variables =  ["topic", "sign"]

# Case 2: topic
#target_variable = "topic"
#categorial_variables =  ["gender", "sign"]

# Case 3: age
#target_variable = "age"
#categorial_variables =  ["topic", "gender", "sign"]

# Case 4: sign
#target_variable = "sign"
#categorial_variables =  ["gender", "topic"]
############################################################################################

# TFIDF params
# use only words that occur at least sqrt_3(X) times 
min_df_exponent = (1/3)


# Iteration for case 1:


# General Preprocessing

This part is independent from the cases

### Initialization 

In [3]:
df = pd.read_csv("../resource/data/blogtext.csv")

# draw random sample for faster processing:
df = df.sample(sample_size)

### Filtering

In [4]:
# filter for a mininmal number of letters in a tweet:
df = df[df["text"].str.count(r"[a-zA-Z]") >= min_chars_per_text]
df = df.reset_index(drop=True)

### Feature Engineering

In [5]:
# append the data
features = [buildFeatures(text) for text  in tqdm(df["text"])]

# merge the features with the original dataset
df_preprocessed = df.merge(pd.DataFrame(features), left_index=True, right_index=True)

100%|██████████| 92/92 [00:00<00:00, 578.88it/s]


### Text Preprocessing

In [6]:
# use the preprocessing  module
preprocessor = Preprocessing.Preprocessing()
df_preprocessed["text_preprocessed"] = preprocessor.ProcessMany(df_preprocessed["text"])

# predict the main language
model = fasttext.load_model('../src/data/lid.176.ftz')
df_preprocessed["main_language"] = [model.predict(text)[0][0].split("__")[-1] for text in df_preprocessed["text_preprocessed"]]

# drop unnecassary features
df_filtered = df_preprocessed[(df_preprocessed["main_language"] == "en")]\
                .drop(["id","text","date","main_language"], axis= 1)

100%|██████████| 92/92 [00:03<00:00, 30.51it/s]


# Individual Transformation

This part is dependent of the cases and need to be done individual for each case

### Transformation

In [7]:
def transformation(X, y, target_variable, categorial_variables, \
                   min_df_exponent, ngram_range, text_features, use_idf):

    # use the  text transformer class to create two transformers for the textual and the numerical model
    text_transformer = TfidfVectorizer(ngram_range=ngram_range, min_df=int(len(X)**(min_df_exponent)), use_idf=use_idf)
    numerical_transformer = make_column_transformer((OneHotEncoder(handle_unknown="ignore"), categorial_variables)\
                                                           , remainder=StandardScaler())

    stacking = StackedTransformation(X, y, numerical_transformer, text_transformer, text_features)
    stacking.build_transformer()
    
    return stacking

### Clustering

In [8]:
def clustering():

    text_data_features = stacking.text_transformer.get_feature_names()
    text_data = stacking.X_train_text_transformed.toarray()

    df_text_cluster = pd.DataFrame(text_data, columns=text_data_features)

    # Die Features beschreiben die Worte im Text
    # Die Werte sind die TF*IDF-transformierten Textdaten

    numerical_data = stacking.X_train_numerical_transformed
    numerical_features = np.append(stacking.numerical_transformer.transformers_[0][1].get_feature_names(),\
                         stacking.X_train_numerical.columns.drop(categorial_variables))

    df_numerical_cluster = pd.DataFrame(numerical_data, columns=numerical_features)
    df_numerical_cluster


    # Die Features mit den x0 - xi Werten beschreiben die Ausprägungen die kategorialen Variablen
    # das jeweilige i beschreibt das i-te Element der im Punkt "Target Variable" definierten liste categorial_variables
    # Die verbleibenden Features (ohne xi) sind Standardskaliert, (x - \mu)/\sigma

    # THIS PART IS STILL MISSING

# Individual Modelling

This part is dependent of the cases and need to be done individual for each case

### Model Selection

In [28]:
class ModelSelection:
    
    def __init__(self, numerical_model, numerical_model_params_dict,\
                 text_model, text_model_params_dict, StackedTransformationInstance):
        
        self.numerical_model = numerical_model
        self.numerical_model_params_dict = numerical_model_params_dict
        
        self.text_model = text_model
        self.text_model_params_dict = text_model_params_dict
        
        self.stacking = StackedTransformationInstance
        self.results = dict()
        
    def GridSearch(self, model, params_dict, X_train, y_train,\
                   X_test, y_test, scoring="accuracy", cv=3):
        
        gs_ = GridSearchCV(model(), scoring=scoring,\
                            param_grid=params_dict, n_jobs=-1, cv=cv, verbose=4)
            
            
        # Use early stopping if model is xgb
        if model == XGBClassifier or  model == XGBClassifier:
            fit_params = {'eval_set': [(X_test, y_test)],
                          'eval_metric': "error",
                          'early_stopping_rounds': 20,
                          'verbose': False}
            
            gs_.fit(X_train, y_train, **fit_params)

        
        else:
            gs_.fit(X_train, y_train)
        
        return {"model": model,
               "params_dict": params_dict,
               "scoring":scoring,
               "cv": cv,
               "grid_search_instance": gs_}
            
    def fit(self, scoring):
        # fit pipeline for numerical model:
        print("Starting Grid search for numerical model ...")
        
        cv_numerical = self.GridSearch(self.numerical_model, self.numerical_model_params_dict,\
                       self.stacking.X_train_numerical_transformed, self.stacking.y_train,\
                       self.stacking.X_test_numerical_transformed, self.stacking.y_test, scoring)
        
        self.results["CV_numerical"] = cv_numerical
        
        # fit pipeline for text model:
        print("Starting Grid search for text model ...")

        
        cv_text = self.GridSearch(self.text_model, self.text_model_params_dict,\
                       self.stacking.X_train_text_transformed, self.stacking.y_train,\
                       self.stacking.X_test_text_transformed, self.stacking.y_test, scoring)
        
        self.results["CV_text"] = cv_text
        
        return self.results
    
def get_cartesian(case_dict):
    case_basic = {"target_type": case_dict["target_type"],
               "target_variable": case_dict["target_variable"],
               "grid_search_metric": case_dict["grid_search_metric"],
               "categorial_variables": case_dict["categorial_variables"]}
    
    product_variables = ["min_df_exponents", "n_gram_range", "use_tfidf", "ml_algorithms_params"]
    variations = product(*[case_dict[var] for var in product_variables])
        
        
    return [{**case_basic, **dict(zip(product_variables, elem))} for elem in variations]

def process_versions(versions_list, X ,y):
    results = []

    for version in tqdm(versions_list):
        if version["target_type"] == "classification":
            le = LabelEncoder()
            y = le.fit_transform(y)

        stacking = transformation(X, y, version["target_variable"], version["categorial_variables"], \
                                version["min_df_exponents"], version["n_gram_range"],\
                                  text_features, version["use_tfidf"])

        ms = ModelSelection(version["ml_algorithms_params"][0], version["ml_algorithms_params"][1],\
                            version["ml_algorithms_params"][0], version["ml_algorithms_params"][1],\
                            stacking)
        
        cv_results = ms.fit(version["grid_search_metric"])
        cv_results = {**cv_results, **version}                            
                            
        results.append(cv_results)
        

    return results

def clean_results(results):
    df = pd.DataFrame(results)
    
    # extract the best parameter together with their errors
    df["CV_numerical_best_params"] = df["CV_numerical"].apply(lambda x: x["grid_search_instance"].best_params_)
    df["CV_numerical_best_score"] = df["CV_numerical"].apply(lambda x: x["grid_search_instance"].best_score_)
    df["CV_text_best_params"] = df["CV_text"].apply(lambda x: x["grid_search_instance"].best_params_)
    df["CV_text_best_score"] = df["CV_text"].apply(lambda x: x["grid_search_instance"].best_score_)
    
    return df

def process(case, df):
    X,y = df.drop(case["target_variable"], axis=1), df[case["target_variable"]]
    versions = get_cartesian(case)
    results = process_versions(versions, X, y)
    return clean_results(results)

In [29]:
# Split the data into  X and y
X,y = df_filtered.drop(target_variable, axis=1), df_filtered[target_variable]

case_gender = {"target_type": "classification",
               "target_variable": "gender",
               "grid_search_metric": "accuracy",
               "text_features": "text_preprocessed",
               "categorial_variables": ["topic","sign"],
               "min_df_exponents" : [(1/4), (1/3), (1/2)],
               "n_gram_range" : [(1,1), (1,2), (2,2)],
               "use_tfidf": [True, False],
               "ml_algorithms_params": [(XGBClassifier,
                                   {'learning_rate': [0.1, 0.8, 1, 1.2],
                                   'max_depth': [7, 8, 9],
                                   'n_estimators': [400, 600, 800]}),
                                  
                                  (SGDClassifier,
                                   {"loss": ["hinge", "log", "modified_huber", "squared_hinge"],
                                   "penalty": ["l2", "l1", "elasticnet"],
                                   "alpha": [0.00001, 0.001]}),
                                  
                                  (LinearSVC,{"penalty": ["l1", "l2"],
                                   "loss": ["hinge", "squared_hinge"],
                                   "C": [0.8, 1, 1.2]})]
              }

                  
versions_gender = get_cartesian(case_gender)
results = process_versions(transformation, versions_gender, X, y)

  0%|          | 0/54 [00:00<?, ?it/s][Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.0s


Starting Grid search for numerical model ...
Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.1s


Starting Grid search for text model ...
Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:    0.5s finished
  2%|▏         | 1/54 [00:00<00:44,  1.20it/s][Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.1s


...

Starting Grid search for text model ...
Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:    0.3s finished
 96%|█████████▋| 52/54 [00:21<00:00,  2.42it/s][Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.0s


Starting Grid search for numerical model ...
Fitting 3 folds for each of 24 candidates, totalling 72 fits
Starting Grid search for text model ...
Fitting 3 folds for each of 24 candidates, totalling 72 fits


[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed:    0.1s finished
 98%|█████████▊| 53/54 [00:22<00:00,  2.67it/s][Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  31 out of  36 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    0.0s finished
100%|██████████| 54/54 [00:22<00:00,  2.43it/s]

Starting Grid search for numerical model ...
Fitting 3 folds for each of 12 candidates, totalling 36 fits
Starting Grid search for text model ...
Fitting 3 folds for each of 12 candidates, totalling 36 fits


### Testing stuff 

In [30]:
df_results = process(case_gender, df_filtered)

  0%|          | 0/54 [00:00<?, ?it/s][Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.1s


Starting Grid search for numerical model ...
Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.1s


Starting Grid search for text model ...
Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:    0.5s finished
  2%|▏         | 1/54 [00:00<00:46,  1.14it/s][Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


...

[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:    0.3s finished
 96%|█████████▋| 52/54 [00:21<00:00,  2.46it/s][Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Starting Grid search for numerical model ...
Fitting 3 folds for each of 24 candidates, totalling 72 fits
Starting Grid search for text model ...
Fitting 3 folds for each of 24 candidates, totalling 72 fits


[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed:    0.1s finished
 98%|█████████▊| 53/54 [00:21<00:00,  2.73it/s][Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  31 out of  36 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    0.0s finished
/home/bigdata/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  31 out of  36 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    0.0s finished
100%|██████████| 54

Starting Grid search for numerical model ...
Fitting 3 folds for each of 12 candidates, totalling 36 fits
Starting Grid search for text model ...
Fitting 3 folds for each of 12 candidates, totalling 36 fits


,CV_numerical,CV_text,target_type,target_variable,grid_search_metric,categorial_variables,min_df_exponents,n_gram_range,use_tfidf,ml_algorithms_params,CV_numerical_best_params,CV_numerical_best_score,CV_text_best_params,CV_text_best_score
0,{'model': <class 'xgboost.sklearn.XGBClassifie...,{'model': <class 'xgboost.sklearn.XGBClassifie...,classification,gender,accuracy,"[topic, sign]",0.250000,"(1, 1)",True,"(<class 'xgboost.sklearn.XGBClassifier'>, {'le...","{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.577778,"{'learning_rate': 1, 'max_depth': 7, 'n_estima...",0.534444
1,{'model': <class 'sklearn.linear_model._stocha...,{'model': <class 'sklearn.linear_model._stocha...,classification,gender,accuracy,"[topic, sign]",0.250000,"(1, 1)",True,(<class 'sklearn.linear_model._stochastic_grad...,"{'alpha': 1e-05, 'loss': 'modified_huber', 'pe...",0.630000,"{'alpha': 1e-05, 'loss': 'hinge', 'penalty': '...",0.563333
2,{'model': <class 'sklearn.svm._classes.LinearS...,{'model': <class 'sklearn.svm._classes.LinearS...,classification,gender,accuracy,"[topic, sign]",0.250000,"(1, 1)",True,"(<class 'sklearn.svm._classes.LinearSVC'>, {'p...","{'C': 1.2, 'loss': 'squared_hinge', 'penalty':...",0.561111,"{'C': 0.8, 'loss': 'squared_hinge', 'penalty':...",0.495556
3,{'model': <class 'xgboost.sklearn.XGBClassifie...,{'model': <class 'xgboost.sklearn.XGBClassifie...,classification,gender,accuracy,"[topic, sign]",0.250000,"(1, 1)",False,"(<class 'xgboost.sklearn.XGBClassifier'>, {'le...","{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.577778,"{'learning_rate': 1.2, 'max_depth': 7, 'n_esti...",0.590556
4,{'model': <class 'sklearn.linear_model._stocha...,{'model': <class 'sklearn.linear_model._stocha...,classification,gender,accuracy,"[topic, sign]",0.250000,"(1, 1)",False,(<class 'sklearn.linear_model._stochastic_grad...,"{'alpha': 1e-05, 'loss': 'hinge', 'penalty': '...",0.616667,"{'alpha': 1e-05, 'loss': 'hinge', 'penalty': '...",0.588889
5,{'model': <class 'sklearn.svm._classes.LinearS...,{'model': <class 'sklearn.svm._classes.LinearS...,classification,gender,accuracy,"[topic, sign]",0.250000,"(1, 1)",False,"(<class 'sklearn.svm._classes.LinearSVC'>, {'p...","{'C': 1.2, 'loss': 'squared_hinge', 'penalty':...",0.561111,"{'C': 1, 'loss': 'squared_hinge', 'penalty': '...",0.494444
6,{'model': <class 'xgboost.sklearn.XGBClassifie...,{'model': <class 'xgboost.sklearn.XGBClassifie...,classification,gender,accuracy,"[topic, sign]",0.250000,"(1, 2)",True,"(<class 'xgboost.sklearn.XGBClassifier'>, {'le...","{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.577778,"{'learning_rate': 1, 'max_depth': 7, 'n_estima...",0.493889
7,{'model': <class 'sklearn.linear_model._stocha...,{'model': <class 'sklearn.linear_model._stocha...,classification,gender,accuracy,"[topic, sign]",0.250000,"(1, 2)",True,(<class 'sklearn.linear_model._stochastic_grad...,"{'alpha': 0.001, 'loss': 'squared_hinge', 'pen...",0.602222,"{'alpha': 0.001, 'loss': 'squared_hinge', 'pen...",0.576667
8,{'model': <class 'sklearn.svm._classes.LinearS...,{'model': <class 'sklearn.svm._classes.LinearS...,classification,gender,accuracy,"[topic, sign]",0.250000,"(1, 2)",True,"(<class 'sklearn.svm._classes.LinearSVC'>, {'p...","{'C': 1.2, 'loss': 'squared_hinge', 'penalty':...",0.561111,"{'C': 1.2, 'loss': 'squared_hinge', 'penalty':...",0.508889
9,{'model': <class 'xgboost.sklearn.XGBClassifie...,{'model': <class 'xgboost.sklearn.XGBClassifie...,classification,gender,accuracy,"[topic, sign]",0.250000,"(1, 2)",False,"(<class 'xgboost.sklearn.XGBClassifier'>, {'le...","{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.577778,"{'learning_rate': 1.2, 'max_depth': 7, 'n_esti...",0.603889
